<a href="https://colab.research.google.com/github/llishyun/deepfake_IZREAL/blob/main/%EA%B3%B5%EB%AA%A8%EC%A0%84_Video_prediction_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# 📌 필요한 패키지 설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install efficientnet-pytorch facenet-pytorch opencv-python-headless scikit-image
!pip install kaggle

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.8 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.6 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.6.0+cu

열지 마세요

In [23]:
# @title
import os
import json

# ✅ Kaggle API Key 직접 지정
kaggle_api_key = {
    "username": "dennyhwan",
    "key": "e6657e203d46afe2c549251f721d4059"
}

# ✅ 경로 설정
kaggle_path = "/root/.kaggle"
kaggle_json_path = os.path.join(kaggle_path, "kaggle.json")

# ✅ kaggle.json 생성 및 저장
os.makedirs(kaggle_path, exist_ok=True)

with open(kaggle_json_path, "w") as f:
    json.dump(kaggle_api_key, f)

# ✅ 권한 설정
os.chmod(kaggle_json_path, 600)

# Youtube 영상 다운로드 받기

In [2]:
!pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 kB 16.4 MB/s eta 0:00:00


In [3]:
#from pytube import YouTube

# Enter the URL of the YouTube video
#video_url = 'https://www.youtube.com/watch?v=xhhGEBf2A7g'

In [4]:
import os
from pytubefix import YouTube

# ✅ YouTube 쇼츠 다운로드 함수
def download_youtube_shorts(url, save_path="downloads"):
    """유튜브 쇼츠(Shorts) 영상을 다운로드하는 함수"""

    # 저장 폴더 생성
    os.makedirs(save_path, exist_ok=True)

    # 유튜브 영상 정보 가져오기
    yt = YouTube(url)
    video_title = yt.title.replace(" ", "_")  # 파일명에 공백 대신 언더스코어 사용

    # 720p 화질의 mp4 스트림 선택 (없는 경우 최적의 화질 선택)
    stream = yt.streams.filter(file_extension="mp4", res="720p").first()
    if stream is None:
        stream = yt.streams.get_highest_resolution()  # 가장 높은 화질로 대체

    # 파일 저장 경로
    save_file = os.path.join(save_path, f"{video_title}.mp4")

    # 다운로드 실행
    stream.download(output_path=save_path, filename=f"{video_title}.mp4")

    print(f"✅ 다운로드 완료: {save_file}")
    return save_file

In [5]:
# ✅ 사용 예제
youtube_url = 'https://www.youtube.com/watch?v=xhhGEBf2A7g'
video_path = download_youtube_shorts(youtube_url)

✅ 다운로드 완료: downloads/내_컴퓨터도_이렇게_돼있으면_무조건_보세요_(느려지는_이유가_있었습니다).mp4


In [6]:
# 유튜브 영상 다운로드 받았다고 가정,,,
# 저장된 동영상 mp4 파일 경로를 video_path라는 변수에 저장.

# Score 계산

In [24]:
import os
import cv2
import numpy as np
import torch
from scipy.special import expit
from skimage.metrics import structural_similarity as ssim

from torch.utils.model_zoo import load_url
from blazeface import BlazeFace
from architectures import fornet, weights
from isplutils import utils
from video_reader import VideoReader
from face_extractor import FaceExtractor

# -----------------------------------------------------
# ✅ 1. 모델 및 얼굴 탐지기 초기화
# -----------------------------------------------------
# 모델 설정
net_model = 'EfficientNetAutoAttB4'
train_db = 'DFDC'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 모델 로드
model_url = weights.weight_url[f'{net_model}_{train_db}']
net = getattr(fornet, net_model)().eval().to(device)
net.load_state_dict(load_url(model_url, map_location=device, check_hash=True))

# 전처리 설정
face_policy = 'scale'
face_size = 224
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

# 얼굴 탐지기 초기화
facedet = BlazeFace().to(device)
facedet.load_weights("blazeface/blazeface.pth")
facedet.load_anchors("blazeface/anchors.npy")

# 비디오 로더
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=32)

# 얼굴 추출기
face_extractor = FaceExtractor(video_read_fn=video_read_fn, facedet=facedet)

ModuleNotFoundError: No module named 'video_reader'

In [7]:
import os
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN
from scipy.special import expit
from efficientnet_pytorch import EfficientNet
from skimage.metrics import structural_similarity as ssim
import torchvision.transforms as transforms

# ✅ **모델 다운로드 (BlazeFace)**
if not os.path.exists("blazeface.pth"):
    !wget https://github.com/hollance/BlazeFace/raw/master/blazeface.pth -O blazeface.pth
if not os.path.exists("anchors.npy"):
    !wget https://github.com/hollance/BlazeFace/raw/master/anchors.npy -O anchors.npy

# ✅ **모델 로드**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
face_detector = MTCNN(keep_all=True, device=device)
net = EfficientNet.from_pretrained('efficientnet-b4').to(device)
net.eval()

# ✅ **전처리 설정 (torchvision 사용)**
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x / 255.0)  # 정규화 (0~1)
])

--2025-03-20 23:08:07--  https://github.com/hollance/BlazeFace/raw/master/blazeface.pth
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-20 23:08:08 ERROR 404: Not Found.

--2025-03-20 23:08:08--  https://github.com/hollance/BlazeFace/raw/master/anchors.npy
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-20 23:08:08 ERROR 404: Not Found.



Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth
100%|██████████| 74.4M/74.4M [00:00<00:00, 99.7MB/s]


Loaded pretrained weights for efficientnet-b4


In [ ]:
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null
%cd icpr2020dfdc/notebook

In [21]:
# 📦 필요 라이브러리 설치 (처음 1회만)
!pip install -q efficientnet_pytorch
!pip install -q imageio[ffmpeg]
!pip install -q scikit-image

# ✅ 모델 다운로드 및 초기화 코드
import torch
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
from scipy.special import expit
import sys
sys.path.append('/content/OpenAVFF')  # 경로 조정

# ✅ 내부 모듈 import (src 구조 기반)
from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet, weights
from isplutils import utils

# ✅ 설정
net_model = 'EfficientNetAutoAttB4'  # 사용할 네트워크
train_db = 'DFDC'                    # 학습된 데이터셋

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 32

# ✅ 모델 다운로드 및 로딩
model_url = weights.weight_url[f'{net_model}_{train_db}']
net = getattr(fornet, net_model)().eval().to(device)
net.load_state_dict(load_url(model_url, map_location=device, check_hash=True))

# ✅ 데이터 전처리 준비
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

# ✅ 얼굴 탐지 모델 준비
facedet = BlazeFace().to(device)
facedet.load_weights("/content/OpenAVFF/blazeface/blazeface.pth")
facedet.load_anchors("/content/OpenAVFF/blazeface/anchors.npy")

# ✅ 비디오 읽기 및 얼굴 추출기
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn, facedet=facedet)

print("✅ 모델, 얼굴탐지기, 전처리 모두 준비 완료!")

Loaded pretrained weights for efficientnet-b4


FileNotFoundError: [Errno 2] No such file or directory: '/content/OpenAVFF/blazeface/blazeface.pth'

In [15]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from scipy.special import expit
from PIL import Image
from facenet_pytorch import MTCNN
from skimage.metrics import structural_similarity as ssim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 🔸 MTCNN 얼굴 탐지기
face_detector = MTCNN(keep_all=False, device=device)

# 🔸 얼굴 판별 모델 불러오기
from efficientnet_pytorch import EfficientNet
net = EfficientNet.from_name('efficientnet-b0')
net.load_state_dict(torch.load('/content/model.pth', map_location=device))
net.eval().to(device)

# 🔸 전처리
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ 얼굴 감지
def detect_faces(video_path, frame_step=10):
    cap = cv2.VideoCapture(video_path)
    face_detected = False
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count > 100:
            break

        if frame_count % frame_step == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces, _ = face_detector.detect(rgb_frame)
            if faces is not None and len(faces) > 0:
                face_detected = True
                break

        frame_count += 1

    cap.release()
    return face_detected

# ✅ 얼굴 기반 탐지
def detect_fake_face(video_path, frame_step=15):
    cap = cv2.VideoCapture(video_path)
    face_scores = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count > 100:
            break

        if frame_count % frame_step == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces, _ = face_detector.detect(rgb_frame)

            if faces is not None and len(faces) > 0:
                x1, y1, x2, y2 = map(int, faces[0])
                h, w, _ = rgb_frame.shape

                x1 = max(0, min(x1, w))
                x2 = max(0, min(x2, w))
                y1 = max(0, min(y1, h))
                y2 = max(0, min(y2, h))

                if x2 - x1 < 5 or y2 - y1 < 5:
                    continue

                face = rgb_frame[y1:y2, x1:x2]
                try:
                    face_tensor = transform(face).unsqueeze(0).to(device)

                    with torch.no_grad():
                        score = net(face_tensor).cpu().numpy().flatten()
                        face_scores.append(expit(score[0]))  # Sigmoid
                except Exception as e:
                    print(f"⚠️ Error: {e}")
        frame_count += 1

    cap.release()
    return np.mean(face_scores) if face_scores else 0.5

# ✅ 얼굴 없는 경우: SSIM 기반
def detect_fake_no_face(video_path, frame_step=10):
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    ssim_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            ssim_score = ssim(prev_frame, frame_gray)
            ssim_scores.append(ssim_score)

        prev_frame = frame_gray
        cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) + frame_step)

    cap.release()
    return 1 - np.mean(ssim_scores)

# ✅ 최종 실행 함수
def predict_video(video_path):
    print(f"🔍 분석 시작: {video_path}")

    if detect_faces(video_path):
        print("🔵 얼굴 감지됨 → 얼굴 기반 AI 탐지")
        score = detect_fake_face(video_path)
    else:
        print("🟠 얼굴 없음 → 비얼굴 AI 탐지")
        score = detect_fake_no_face(video_path)

    fscore = 1 - score
    print(f"🔹 Real Score: {score:.6f}")
    print(f"🔹 Fake Score: {fscore:.6f}")
    print("🟢 판별:", "Real" if score > 0.5 else "Fake")


FileNotFoundError: [Errno 2] No such file or directory: '/content/model.pth'

In [13]:
# 🚀 6. 최종 실행 함수
def predict_video(video_path):
    print(f"🔍 분석 시작: {video_path}")

    if detect_faces(video_path):
        print("🔵 얼굴 감지됨 → 얼굴 기반 AI 탐지")
        score = detect_fake_face(video_path)
    else:
        print("🟠 얼굴 없음 → 비얼굴 AI 탐지")
        score = detect_fake_no_face(video_path)
    fscore = 1 - score
    print(f"🔹 Score for real video: {score:.6f}")
    print(f"🔹 Score for fake video: {fscore:.6f}")

    if score > 0.5:
      print("Real입니다.")
    else:
      print("Fake입니다.")

In [14]:
video_path = input("🎥 유튜브 영상 링크를 입력하세요: ")
predict_video(video_path)

🎥 유튜브 영상 링크를 입력하세요: https://www.youtube.com/watch?v=I4Opm59JFT4
🔍 분석 시작: https://www.youtube.com/watch?v=I4Opm59JFT4
🟠 얼굴 없음 → 비얼굴 AI 탐지
🔹 Score for real video: 0.500000
🔹 Score for fake video: 0.500000
Fake입니다.


In [ ]:
https://www.youtube.com/watch?v=I4Opm59JFT4